In [1]:
import numpy as np
import json
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
from scipy import stats
from os.path import exists
import os
from tqdm import tqdm
from itertools import combinations, product


from concurrent.futures import ProcessPoolExecutor
import os
import mmap
import ast


In [2]:
def process_directory(directory, obs_dict):
    filenames = [f for f in os.listdir(directory) if f.startswith('rtt')]
    for path in tqdm(filenames):
        path_st = path.replace('rtt', 'state')
        collect_data_per_txt_with_approx(
            os.path.join(directory, path),
            os.path.join(directory, path_st), 
            obs_dict
        )

In [25]:
def collect_data_txt():
    obs_dict = [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
    directories = ['control/data_pid/1_loop', 'control/data_pid/2_loop', 'control/data_pid/3_loop', 'control/data_pid/4_loop', 'control/data_pid/5_loop']#, 'dataset/data_corrected/datar5/']
    directories = ['control/data_lqg/1_loop', 'control/data_lqg/2_loop', 'control/data_lqg/3_loop', 'control/data_lqg/4_loop']#, 'dataset/data_corrected/datar5/']

    
    for directory in directories:
        filenames = [f for f in os.listdir(directory) if f.startswith('rtt')]
        cnt = 0
        for path in tqdm(filenames):
            cnt += 1
            path_st = path.replace('rtt', 'state')
            # Create a copy of the dictionary for each file processing
            local_dict = [{k: v.copy() for k, v in d.items()} for d in obs_dict]
            collect_data_per_txt_with_approx(
                os.path.join(directory, path),
                os.path.join(directory, path_st), 
                local_dict
            )
            # Update the main dictionary after processing
            for i in range(len(obs_dict)):
                obs_dict[i].update(local_dict[i])
            if cnt == 1000:
                break
    
    return obs_dict

In [4]:
N_acceptance = 10

In [19]:

            
def collect_data_per_txt_with_approx(path_rtt, path_state, obs_dict):
    # Read the files using numpy's genfromtxt for more efficient reading
    data_rtt = np.genfromtxt(path_rtt, delimiter=', ', dtype=float)
    data_state = np.genfromtxt(path_state, delimiter=', ', dtype=float)

    # Vectorize the acceptance calculation
    acceptances = []
    states = []
    bws = []
    for t in range(len(data_rtt) - N_acceptance):
        lst = data_rtt[t:t+N_acceptance]

        acc = np.where(lst == 0, 0, 
              np.where((lst > 10 * (N_acceptance - np.arange(N_acceptance))) | 
                      ((lst == -1)), 4,
                      np.where(lst == -1, 2, 1)))
        acceptances.append(acc.astype(np.int32))
        
    for t in range(N_acceptance + 1, len(data_rtt)):
        lst = data_rtt[max(0, t - 100):t]
      #  acc = np.where(lst == 0, 0, 
       #               np.where((lst > 10 * (N_acceptance - np.arange(N_acceptance))) | 
        #                      ((lst == -1)), 4))
        sum_data = np.sum(np.array(lst) > 0)
        sum_delay = np.sum(np.array(lst)[np.where(np.array(lst) > 0)])
        #print(sum_data, sum_delay, np.round(sum_data/sum_delay*100))
        if sum_delay:
            bws.append(np.round(sum_data/sum_delay*100))
        else:
            bws.append(0)
   # print(acceptances)
    # Vectorize state processing
    states = np.clip(data_state[N_acceptance + 1:], -2000, 2000).astype(np.float16)

    # Process dictionary updates
    for t in range(min(len(acceptances), len(states))-1):
        if 0.0 not in states[t-10:t+1]:
            key = str(list(acceptances[t])[-10:] + [quantize_to_grid(states[t])] + [bws[t]])
            val = str(list(acceptances[t+1]) + [quantize_to_grid(states[t+1])] + [bws[t+1]])
            if states[t+1] == 0:
                print(val)
            
            if key not in obs_dict[9]:
                obs_dict[9][key] = []
            obs_dict[9][key].append(val)
            #print(key, val)
            key = str(list(acceptances[t])[-9:] + [quantize_to_grid(states[t])] + [bws[t]])
            if key not in obs_dict[8]:
                obs_dict[8][key] = []
            obs_dict[8][key].append(val)

In [7]:
def quantize_to_grid(number):
    # Define the grid for numbers up to 600
    grid = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0,
            1.2, 1.4, 1.7, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10,
            12, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100,
            120, 150, 200, 250, 300, 350, 400, 450, 500, 600]
    
    # Handle negative numbers
    if number < 0:
        return -quantize_to_grid(abs(number))
    
    # For numbers larger than 600, round to nearest hundred
    if number > 600:
        return np.round(round(number / 100) * 100, 1)
    if number == 0:
        return 2000
    
    # Find the closest value in the grid
    return min(grid, key=lambda x: abs(x - number))

In [26]:
collected = collect_data_txt()

100%|███████████████████████████████████████████| 40/40 [01:40<00:00,  2.51s/it]


In [ ]:
collected1 = collect_data_txt()

  1%|          | 10/1500 [00:11<26:16,  1.06s/it]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2000]


  1%|          | 17/1500 [00:18<27:45,  1.12s/it]

[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 2000]


  2%|▏         | 23/1500 [00:25<28:07,  1.14s/it]

...

 12%|█▏        | 253/2125 [26:49<3:33:39,  6.85s/it]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2000]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2000]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2000]
[0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 2000]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2000]


 12%|█▏        | 254/2125 [26:56<3:39:43,  7.05s/it]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2000]
[4, 0, 0, 4, 0, 0, 0, 4, 0, 0, 2000]
[0, 4, 0, 4, 4, 0, 4, 4, 0, 0, 2000]
[0, 4, 4, 4, 4, 4, 0, 0, 4, 4, 2000]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2000]
[0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 2000]


 12%|█▏        | 255/2125 [27:02<3:25:49,  6.60s/it]

In [15]:
import ast
from collections import Counter
import math
from tqdm import tqdm

def modify_dict(input_dict):
    def truncate_list(value_list, target_length):
        # Convert strings to tuples of numeric values
        numeric_tuples = [tuple(ast.literal_eval(s)) for s in value_list]
        
        # Count occurrences of each unique tuple
        counter = Counter(numeric_tuples)
        
        # Calculate the total count
        total_count = sum(counter.values())
        
        # Calculate the scaling factor to reduce to target length
        scaling_factor = target_length / total_count
        
        # Create a new list with scaled occurrences
        new_list = []
        for item, count in counter.items():
            # Calculate the new count, ensuring at least 1 occurrence
            new_count = max(1, math.floor(count * scaling_factor))
            new_list.extend([item] * new_count)
        
        # Trim the list to exactly target length
        if len(new_list) > target_length:
            new_list = new_list[:target_length]
        
        # Convert items back to strings
        return [str(list(item)) for item in new_list]

    for key, value_list in tqdm(input_dict.items()):
        # Separate lists based on 10th element
        list_with_0 = [item for item in value_list if ast.literal_eval(item)[-3] == 0]
        list_with_4 = [item for item in value_list if ast.literal_eval(item)[-3] == 4]
        #print(len(list_with_0), len(list_with_4))
        
        # Truncate if necessary
        if len(list_with_0) > 99:
            list_with_0 = truncate_list(list_with_0, 99)
            #print(list_with_0)
        if len(list_with_4) > 99:
            list_with_4 = truncate_list(list_with_4, 99)
        
        # Merge the lists
        merged_list = list_with_0 + list_with_4
       # print(len(merged_list))

        
        # Update the dictionary
        input_dict[key] = merged_list

    return input_dict


In [27]:
col8_upd = modify_dict(collected[8])

100%|█████████████████████████████████| 220013/220013 [00:26<00:00, 8451.41it/s]


In [65]:
col18_upd = modify_dict(collected1[8])

100%|██████████| 223902/223902 [04:46<00:00, 780.25it/s]  


In [28]:
col9_upd = modify_dict(collected[9])

100%|████████████████████████████████| 276042/276042 [00:21<00:00, 12847.96it/s]


In [66]:
col19_upd = modify_dict(collected1[9])

100%|██████████| 406720/406720 [04:42<00:00, 1438.30it/s] 


In [18]:
def save_dict(dictionary, filename):
    with open(filename, 'w') as f:
        json.dump(dictionary, f)

In [29]:
save_dict(col8_upd, "9bwlqg_observations_txt.json")

In [30]:
save_dict(col9_upd, "10bwlqg_observations_txt.json")

In [175]:
save_dict(col8_upd, "9bwpid_observations_txt.json")

In [176]:
save_dict(col9_upd, "10bwpid_observations_txt.json")